In [44]:
#pip install geoalchemy2

Note: you may need to restart the kernel to use updated packages.


In [7]:
#!pip install openpyxl

   ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/250.0 kB ? eta -:--:--
   - -------------------------------------- 10.2/250.0 kB ? eta -:--:--
   --------- ----------------------------- 61.4/250.0 kB 656.4 kB/s eta 0:00:01
   ------------------------------------ --- 225.3/250.0 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 250.0/250.0 kB 1.7 MB/s eta 0:00:00


In [1]:
import sys
import pandas as pd
import numpy as np
import time, os
from sqlalchemy import create_engine, event, schema, Enum, Table, text, Null, Column, String, Date, Float, Integer, MetaData, CheckConstraint, ForeignKeyConstraint, CheckConstraint, VARCHAR, NUMERIC, PrimaryKeyConstraint, ForeignKey
from geoalchemy2 import Geography, Geometry
from sqlalchemy_utils import database_exists, create_database
from sqlalchemy.orm import sessionmaker
import psycopg2
from psycopg2 import Error
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Here you define the credentials
DATABASE = "" # TO BE REPLACED
USER = "" # TO BE REPLACED
PASSWORD = "" # TO BE REPLACED
HOST = "" 
PORT = ""

# Our connection to the database
try :
    connection = psycopg2.connect (
        database = DATABASE ,
        user = USER ,
        password = PASSWORD ,
        host ='dbcourse.cs.aalto.fi',
        port = '5432'
)
    connection.autocommit = True 

except ( Exception , Error ) as error : # In case we fail to establish the connection
    print (" Error while connecting to PostgreSQL ", error )

# Create the database URL
db_url = f'postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}'

# Create an engine instance
engine = create_engine(db_url)

In [6]:
metadata = MetaData()

city = Table(
    'city', metadata,
    Column('name', VARCHAR(15), nullable = False),
    Column('id', VARCHAR(3), primary_key = True)
)

volunteer = Table(
    'volunteer', metadata,
    Column('id',VARCHAR(11), primary_key = True),
    Column('birthdate', Date, nullable = False),
    Column('city_id', VARCHAR(3), ForeignKey('city.id'), nullable= False),
    Column('name', VARCHAR(50), nullable = False),
    Column('email', VARCHAR(50), nullable = False),
    Column('address', VARCHAR(50), nullable = False),
    Column('travel_readiness', Integer, nullable = False),
)


volunteer_range = Table(
    'volunteer_range', metadata,
    Column('volunteer_id', VARCHAR(11), ForeignKey('volunteer.id')),
    Column('city_id', VARCHAR(3), ForeignKey('city.id')),
    PrimaryKeyConstraint('volunteer_id','city_id')
)

skill = Table(
    'skill', metadata,
    Column('name', VARCHAR(50), primary_key = True),
    Column('description', VARCHAR)
)

skill_assignment = Table(
    'skill_assignment', metadata,
    Column('volunteer_id', VARCHAR(11), ForeignKey('volunteer.id')),
    Column('skill_name', VARCHAR(50), ForeignKey('skill.name')),
    PrimaryKeyConstraint('volunteer_id', 'skill_name')
)

interest = Table(
    'interest', metadata, 
    Column('name', VARCHAR(50), primary_key = True)
)

interest_assignment = Table(
    'interest_assignment', metadata,
    Column('interest_name', VARCHAR(50), ForeignKey('interest.name')),
    Column('volunteer_id', VARCHAR(11), ForeignKey('volunteer.id')),
    PrimaryKeyConstraint('interest_name', 'volunteer_id')
)

beneficiary = Table(
    'beneficiary', metadata, 
    Column('id', VARCHAR, primary_key = True),
    Column('name', VARCHAR(50), nullable = False),
    Column('address', VARCHAR(50), nullable = False),
    Column('city_id', VARCHAR(3), ForeignKey('city.id'), nullable = False)
)

request = Table(
    'request', metadata, 
    Column('id', VARCHAR, primary_key = True),
    Column('beneficiary_id', VARCHAR, ForeignKey('beneficiary.id'), nullable = False),
    Column('number_of_volunteers', Integer, nullable = False),
    Column('priority_value', VARCHAR, nullable = False),
    Column('start_date', Date, nullable = False),
    Column('end_date', Date, nullable = False),
    Column('register_by_date', Date, nullable = False)
)

request_skill = Table(
    'request_skill', metadata,
    Column('request_id', VARCHAR, ForeignKey('request.id')),
    Column('skill_name', VARCHAR, ForeignKey('skill.name')),
    Column('min_need', Integer, nullable = False),
    Column('value', Integer, nullable = False),
    PrimaryKeyConstraint('request_id', 'skill_name')
)

request_location = Table(
    'request_location', metadata,
    Column('request_id', VARCHAR, ForeignKey('request.id')),
    Column('city_id', VARCHAR(3), ForeignKey('city.id')),
    PrimaryKeyConstraint('request_id', 'city_id')
)

volunteer_application = Table(
    'volunteer_application', metadata,
    Column('request_id', VARCHAR, ForeignKey('request.id')),
    Column('volunteer_id', VARCHAR(11), ForeignKey('volunteer.id')),
    PrimaryKeyConstraint('request_id', 'volunteer_id')
)

metadata.create_all(engine)

In [7]:
query = """
   ALTER TABLE city
   ADD geolocation POINT;
"""
with connection.cursor() as cur:
    cur.execute(query)

In [26]:
query = """
   INSERT INTO city VALUES ('Tinh', '1', POINT(1.2, 3.4));
"""
with connection.cursor() as cur:
    cur.execute(query)

In [33]:
query = """
    Select * from city
"""
test = pd.read_sql_query(query, engine.connect()) 
test.head()

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "city" does not exist
LINE 2:     Select * from city
                          ^

[SQL: 
    Select * from city
]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [16]:
query = """
   Drop all
"""
with connection.cursor() as cur:
    cur.execute(query)

In [32]:
query = "SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';"
with connection.cursor() as cur:
    cur.execute(query)
    tables = cur.fetchall()
    for table in tables:
        drop_query = f"DROP TABLE {table[0]} CASCADE;"
        cur.execute(drop_query)
    connection.commit()

In [8]:
data = pd.ExcelFile('data.xlsx')

In [11]:
data.sheet_names

['city',
 'volunteer',
 'volunteer_range',
 'skill',
 'skill_assignment',
 'interest',
 'interest_assignment',
 'beneficiary',
 'request',
 'request_skill',
 'request_location',
 'volunteer_application']

In [41]:
df = data.parse('city')